# Spreadsheet columns
- Publisher - org displaying the ad
- Advertiser - brand paying for ad
- Campaign - marketing initiative
- Imps - number of times displayed regardless of views
- Viewable imps - times ad is visible on screen
- Clicks - num clicks
- Dsp total cost USD - cost for advertiser to place ads along with platform fees etc
- Dsp media cost USD - cost for advertiser to place ads
- Ssp media cost USD - fee (%) taken by publisher on any revenue generated by ads
    - is this separate to display side platform costs?
- Pc convs - conversions after click
- Total convs - conversions
- Adstxt verified imps - confirmed seen by real person

In [16]:
import numpy as np
import pandas as pd

In [17]:
dataset = "../data/dataset.csv"
df = pd.read_csv(dataset, index_col=None, thousands=',')
df.describe()

,Imps,Viewable Imps,Clicks,Dsp Total Cost USD,Dsp Media Cost USD,Ssp Media Cost USD,Pc Convs,Total Convs,Adstxt Verified Imps
count,79065.000000,79065.000000,79065.000000,79065.000000,79065.000000,79065.000000,79065.000000,79065.000000,79065.000000
mean,939.322836,511.779473,1.468539,2.252957,1.855104,1.424515,0.074470,2.276393,499.851173
std,3589.117932,2151.531299,6.794143,6.757863,5.543096,4.235450,0.923525,22.476481,1997.382888
min,1.000000,0.000000,0.000000,0.200000,0.010000,0.010000,0.000000,0.000000,0.000000
25%,13.000000,0.000000,0.000000,0.310000,0.250000,0.190000,0.000000,0.000000,7.000000
50%,82.000000,0.000000,0.000000,0.550000,0.450000,0.350000,0.000000,0.000000,46.000000
75%,346.000000,99.000000,0.000000,1.580000,1.310000,1.020000,0.000000,0.000000,236.000000
max,159220.000000,93776.000000,320.000000,405.370000,365.640000,292.510000,44.000000,1222.000000,108361.000000


# Set dtypes

In [18]:
df["Datetime"] = pd.to_datetime(df["Datetime"])
df["Publisher"] = df["Publisher"].astype("category")
df["Advertiser"] = df["Advertiser"].astype("category")
df["Campaign"] = df["Campaign"].astype("category")


# Convert datetime to day of week and hour of day

In [19]:
day, hour = df["Datetime"].dt.dayofweek, df["Datetime"].dt.hour

df["Day of week"] = day.astype("category")
df["Hour of day"] = hour.astype("category")

# Add verified impression rate

In [20]:
df["VerifiedImpRate"] = df["Adstxt Verified Imps"].astype(int) / df["Imps"].astype(int)

# Add conversion / verified impression rate

In [21]:
df["ConvPerVerImp"] = df["Total Convs"].astype(int) / df["Adstxt Verified Imps"].astype(int)

### Ssp rates

In [22]:
df["SspCostPerConv"] = df["Ssp Media Cost USD"].astype(float) / df["Total Convs"].astype(int)

### Dsp rates

In [23]:
# Dsp total cost per conversion
df["DspTotalCostPerConv"] = df["Dsp Total Cost USD"].astype(float) / df["Total Convs"].astype(int)

# Dsp total cost per verified impression
df["DspTotalCostPerVerImp"] = df["Dsp Total Cost USD"].astype(float) / df["Adstxt Verified Imps"].astype(int)

# Overall rates

In [24]:
# Conversions per usd
df["ConversionsPerUSD"] = df["Total Convs"].astype(float) / (df["Dsp Total Cost USD"].astype(float))
# verified impressions per usd
df["VerImpPerUSD"] = df["Adstxt Verified Imps"].astype(float) / (df["Dsp Total Cost USD"].astype(float))

# Handle infinite values and save csv

In [25]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.to_csv("../data/engineered-dataset.csv", index=False)

# Finding key insights

In [26]:
# top campaign / publisher combo
df_brand_b = df[df["Advertiser"] == "Brand B"]

campaign_performance = df_brand_b.groupby(["Publisher", "Campaign"])["ConversionsPerUSD"].mean().reset_index()
campaign_performance_sorted = campaign_performance.sort_values(by="ConversionsPerUSD", ascending=False)

print(campaign_performance_sorted.head())


                   Publisher Campaign  ConversionsPerUSD
18102              SofaScore      B18         149.666582
15480          Perform Group      B18         127.607487
7683            Futbol Sites      B18          80.823139
21000  Undisclosed publisher      B18          63.428101
6855              Fandom Inc      B18          55.872926


C:\Users\alex0\AppData\Local\Temp\ipykernel_179800\561660843.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  campaign_performance = df_brand_b.groupby(["Publisher", "Campaign"])["ConversionsPerUSD"].mean().reset_index()


In [27]:
campaign_b18 = df[df["Campaign"] == "B18"]

interaction_counts = {
    "SofaScore-B18": campaign_b18[campaign_b18["Publisher"] == "SofaScore"].shape[0],
    "Perform Group-B18": campaign_b18[campaign_b18["Publisher"] == "Perform Group"].shape[0],
    "Futbol Sites-B18": campaign_b18[campaign_b18["Publisher"] == "Futbol Sites"].shape[0]
}

print("Number of samples for each interaction:")
for interaction, count in interaction_counts.items():
    print(f"{interaction}: {count}")

Number of samples for each interaction:
SofaScore-B18: 103
Perform Group-B18: 52
Futbol Sites-B18: 28
